In [141]:
import requests
import pandas as pd

In [142]:
url = "https://api.frankfurter.app/latest?from=EUR"

In [143]:
response = requests.get(url)
data = response.json()

In [144]:
data

{'amount': 1.0,
 'base': 'EUR',
 'date': '2025-06-13',
 'rates': {'AUD': 1.7777,
  'BGN': 1.9558,
  'BRL': 6.4007,
  'CAD': 1.569,
  'CHF': 0.9359,
  'CNY': 8.2682,
  'CZK': 24.831,
  'DKK': 7.4586,
  'GBP': 0.8505,
  'HKD': 9.0362,
  'HUF': 403.43,
  'IDR': 18784,
  'ILS': 4.1422,
  'INR': 99.11,
  'ISK': 144.2,
  'JPY': 165.94,
  'KRW': 1577.42,
  'MXN': 21.881,
  'MYR': 4.8874,
  'NOK': 11.4435,
  'NZD': 1.9167,
  'PHP': 64.706,
  'PLN': 4.2745,
  'RON': 5.033,
  'SEK': 10.9635,
  'SGD': 1.4777,
  'THB': 37.351,
  'TRY': 45.397,
  'USD': 1.1512,
  'ZAR': 20.736}}

In [145]:
df_rates = pd.DataFrame(data["rates"].items(), columns=["target_currency", "rate"])

In [146]:
df_rates["amount"] = data.get("amount")
df_rates["base_currency"] = data.get("base")
df_rates["date"] = data.get("date")

In [147]:
df_final = df_rates[["amount", "base_currency", "date", "target_currency", "rate"]]

In [148]:
df_final = df_final.dropna(subset=["amount", "base_currency", "date", "target_currency", "rate"])

In [149]:
from sqlalchemy import create_engine

In [150]:
db_name = "currency_etl"
db_user = "postgres"
db_password = "postgre"
db_host = "localhost"
db_port = 5432

In [151]:
db_url = "postgresql://{0}:{1}@{2}:{3}/{4}".format(
    db_user, db_password, db_host, db_port, db_name)
engine = create_engine(db_url)

In [152]:
from sqlalchemy import text

In [153]:
with engine.connect() as conn:
    for index, row in df_final.iterrows():
        query = text("""
        INSERT INTO exchange_rates (amount, base_currency, date, target_currency, rate)
        VALUES (:amount, :base_currency, :date, :target_currency, :rate)
        ON CONFLICT (base_currency, target_currency, date) DO NOTHING;
        """)
        conn.execute(query, row.to_dict())
    conn.commit()

In [154]:
df_final

,amount,base_currency,date,target_currency,rate
0,1.0,EUR,2025-06-13,AUD,1.7777
1,1.0,EUR,2025-06-13,BGN,1.9558
2,1.0,EUR,2025-06-13,BRL,6.4007
3,1.0,EUR,2025-06-13,CAD,1.5690
4,1.0,EUR,2025-06-13,CHF,0.9359
5,1.0,EUR,2025-06-13,CNY,8.2682
6,1.0,EUR,2025-06-13,CZK,24.8310
7,1.0,EUR,2025-06-13,DKK,7.4586
8,1.0,EUR,2025-06-13,GBP,0.8505
9,1.0,EUR,2025-06-13,HKD,9.0362
